In [1]:
import torch
from mmdet.structures.mask.structures import BitmapMasks
import torch.nn.functional as F
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

In [2]:
gt_instance_mask_labels = torch.load('gt_instance_mask_labels.pt',map_location=torch.device('cpu'))
gt_instance_masks = torch.load('gt_instance_masks.pt',map_location=torch.device('cpu'))
seg_fields = torch.load('seg_fields.pt',map_location=torch.device('cpu'))
seg_preds = torch.load('seg_preds.pt',map_location=torch.device('cpu'))

In [3]:
labels = gt_instance_mask_labels
masks = gt_instance_masks
fields = seg_fields
preds = seg_preds
thinglabels={2,3,4,5,8,9,10,11}
tmasks = [mask.to_tensor(dtype=torch.float32, device='cpu') for mask in masks]
nmasks = [mask.to_ndarray() for mask in masks]

In [4]:
def visualize_tensor(tensor):
    # Convert the PyTorch tensor to a NumPy array
    array = tensor.numpy()

    # Display the array using Matplotlib
    plt.imshow(array, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.show()

In [5]:
def visualize_array(array):
    plt.imshow(array, cmap='viridis')
    plt.colorbar()
    plt.show()

In [6]:
def save_array_as_png(array, file_path):
    # Normalize the array to 0-255
    normalized_array = (array - np.min(array)) / (np.max(array) - np.min(array))
    scaled_array = (normalized_array * 255).astype(np.uint8)

    # Create PIL Image from the array
    image = Image.fromarray(scaled_array)

    # Save the image as PNG
    image.save(file_path)

In [8]:
def getcenterfield(mask):
    # Calculate the bounding box around the mask
    positive_indices = np.where(mask)
    min_row, min_col = np.min(positive_indices, axis=1)
    max_row, max_col = np.max(positive_indices, axis=1)
    center = ((min_col + max_col) // 2, (min_row + max_row) // 2)
    width = max_col - min_col + 1
    height = max_row - min_row + 1

    # Generate the Gaussian kernel
    deviation = (width /3, height / 3)
    def generate_2d_gaussian_kernel(shape, center, deviation):
        x, y = np.indices(shape)
        y_center, x_center = center
        exponent = -((x - x_center)**2 / (2 * deviation[1]**2) + (y - y_center)**2 / (2 * deviation[0]**2))
        gaussian_kernel = np.exp(exponent)
        return gaussian_kernel
    kernel = generate_2d_gaussian_kernel(mask.shape, center, deviation)
    filtered_mask = kernel * mask
    return filtered_mask

def getdistancefield(mask):
    # Find the indices of positive elements in the mask
    positive_indices = np.where(mask)

    # Calculate the bounding box around the positive elements
    min_row, min_col = np.min(positive_indices, axis=1)
    max_row, max_col = np.max(positive_indices, axis=1)

    # Calculate the center point of the bounding box
    center_row = (min_row + max_row) // 2
    center_col = (min_col + max_col) // 2

    # Calculate the width and height of the bounding box
    width = max_col - min_col + 1
    height = max_row - min_row + 1

    # Calculate the coordinates of each pixel in the image
    rows, cols = np.indices(mask.shape[:2])

    # Calculate the distances between each pixel and the center point
    distances_x = (cols - center_col) / (width - 1)
    distances_y = (rows - center_row) / (height - 1)

    # Stack the distances along the channel axis
    distances = np.stack((distances_x, distances_y), axis=-1)
    masked_tensor = distances * mask[..., None]
    return masked_tensor

def getfields(mask):
    distancefield = getdistancefield(mask)
    centerfield = getcenterfield(mask)
    return np.concatenate((distancefield, centerfield[..., None]), axis=-1)

In [9]:
def batch_getfields(batched_masks):
    batch_fields = []
    for masks in batched_masks:
        fields = getfields(masks[0])
        for mask in masks[1:]:
            fields += getfields(mask)
        batch_fields.append(fields)
    return batch_fields

In [34]:
def batch_getfields_classwise(bitmap_batch,tensorlabels):
    batched_masks = [mask.to_ndarray() for mask in bitmap_batch]
    labels = [label.numpy() for label in tensorlabels]
    batch_fields = []
    out_tensors = []
    for labels,masks in zip(labels,batched_masks):
        dummy = getfields(masks[0])*0
        fields = {i:torch.from_numpy(dummy.copy()) for i in thinglabels}
        for label,mask in zip(labels,masks):
            fields[int(label)] += torch.from_numpy(getfields(mask))
        batch_fields.append(fields)
        out_tensors.append(torch.cat(tuple(fields.values()),dim = -1).permute([2,0,1]))
    return batch_fields,torch.stack(out_tensors,dim=0)

In [35]:
allm,allt = batch_getfields_classwise(masks,labels)

In [39]:
seg_fields.shape

torch.Size([2, 24, 160, 336])

In [17]:
def visualize_rgb_array(array):
    # Visualize each channel separately
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    for i in range(3):
        im = axes[i].imshow(array[..., i], cmap='gray')
        axes[i].set_title(f'Channel {i+1}')
        fig.colorbar(im, ax=axes[i])

    plt.show()

In [ ]:
visualize_rgb_array(allt[0][2])

COPIED CODE

In [ ]:
# Bura benden
        # Device degistir oyle dene sorun cozulmedi
        """
        c = 0
        vs = []
        for bgm in gt_masks:
            bb = bgm.get_bboxes()
            #vfh = torch.tensor(bgm[0].masks[0],dtype=torch.float32, device=gt_bboxes[0].device)
            #vfw = torch.tensor(bgm[0].masks[0],dtype=torch.float32, device=gt_bboxes[0].device)
            vfh = torch.zeros(bgm[0].masks[0].shape,dtype=torch.float32, device='cpu')
            vfw = torch.zeros(bgm[0].masks[0].shape,dtype=torch.float32, device='cpu')
            for i,gm in enumerate(bgm):
                bbox = bb[i]
                cbbw = (bbox[0]+bbox[2])/2
                cbbh = (bbox[1]+bbox[3])/2
                w = bbox[2] - bbox[0]
                h = bbox[3] - bbox[1]
                #m = torch.tensor(gm,dtype=torch.float32, device=gt_bboxes[0].device)
                m = torch.tensor(gm,dtype=torch.float32, device='cpu')
                vh = m.clone()
                vw = m.clone()
                for ih in range(m.shape[0]):
                    for iw in range(m.shape[1]):
                        if m[ih][iw]:
                            #vh[ih][iw] = (ih - cbbh)/h
                            #vw[ih][iw] = (iw - cbbw)/w
                            vh[ih][iw] = (ih - cbbh)
                            vw[ih][iw] = (iw - cbbw)
                vfh.add_(vh)
                vfw.add_(vw)
            vs.append([vfh,vfw])
        vs
        from debug import yukle,yuklemask,gor2d,gor2dabs
        from mmcv.visualization import imshow_bboxes
        yukle(img)
        yuklemask(gt_masks)
        imshow_bboxes('sil.png',gt_bboxes[0].cpu().numpy(), out_file='sil2.png', show=False)
        gor2d(vs[0][0])
        gor2dabs(vs[0][0])
        """

        gt_semantic_segs = torch.stack(gt_semantic_segs)
        if self.seg_rescale_factor != 1.0:
            gt_semantic_segs = F.interpolate(
                gt_semantic_segs.float(),
                scale_factor=self.seg_rescale_factor,
                mode='nearest').squeeze(1)

        # Things classes will be merged to one class in PanopticFPN.
        gt_semantic_segs = self._set_things_to_void(gt_semantic_segs)

        if seg_preds.shape[-2:] != gt_semantic_segs.shape[-2:]:
            seg_preds = interpolate_as(seg_preds, gt_semantic_segs)
        seg_preds = seg_preds.permute((0, 2, 3, 1))

        loss_seg = self.loss_seg(
            seg_preds.reshape(-1, self.num_classes),  # => [NxHxW, C]
            gt_semantic_segs.reshape(-1).long())

        return dict(loss_seg=loss_seg)